# Street View House Numbers - Indexed Search

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import svhn
import graphics
import keras_utils
from keras.utils import np_utils

%matplotlib inline

# define some constants
max_digits = 7
image_size = (32,77)
checkpoint_path = '../checkpoints/model.hdf5'
resume_training = True

# print the keras version used
import keras
print "Keras version : {}".format(keras.__version__)

Using TensorFlow backend.


Keras version : 1.2.0


## Generate data
For training, we will use a flattened dataset, where each training sample represents one digit. The inputs per sample will be resized image and the index of digit which needs to be detected, and outputs will be the number of digits in the image and the digit at given index.

Because of flattening the dataset, the training set for label detector will be expanded, while the one for counter will have repeated samples. So in a way, the label detector will have more data to train with, but the counter can be assumed to be trained for more epochs.

change **nsamples** below to **33704** to train with full dataset

In [2]:
# read the h5py data file (takes time)
rawdata = svhn.read_process_h5('../inputs/train/digitStruct.mat')

In [3]:
nsamples = 1000

def generateData(data, n=1000):
    '''
    generates flattened SVHN dataset
    '''
    Ximg_flat = []
    Xidx_flat = []
    ycount_flat = []
    ycoord_flat = []
    ylabel_flat = []
    
    for datapoint in np.random.choice(data, size=n):
        img,_ = svhn.createImageData(datapoint, image_size, '../inputs/train/')
        for i in range(0,datapoint['length']):
            Ximg_flat.append(img)
            Xidx_flat.append(i)
            ycount_flat.append(datapoint['length'])
            ylabel_flat.append(datapoint['labels'][i])
            
    ylabel_flat = [0 if y==10 else int(y) for y in ylabel_flat]
    return np.array(Ximg_flat), np.array(Xidx_flat), np.array(ycount_flat), np.array(ylabel_flat)

Ximg, Xidx, ycount, ylabel = generateData(rawdata, nsamples)

## Create Deep Learning Network
Our deep learning network consists of a shared vision model, a counter and a label detector.

The vision model processes input image of fixed size using Convolutional Neural Networks and produces a dense tensor of shape (1024,). This tensor is then processed by a counter which is made of fully connected layers. The counter output is used to generates indices, which are combined with the vision model output and fed to label detector, which outputs a label for each index.

In [4]:
from keras.layers import Input, Dense, Merge, Flatten, Dropout, merge
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model, Sequential

# define vision model
image_in_vision = Input(shape=(image_size[0],image_size[1],3))
x = Convolution2D(32, 3, 3, activation='tanh')(image_in_vision)
x = Convolution2D(32, 3, 3, activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Convolution2D(128, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
h = Dense(1024, activation='relu')(x)
vision_model = Model(input=image_in_vision, output=h, name='vision')

In [5]:
# define counter model
h_in_counter = Input(shape=(1024,))
yc = Dense(512, activation='relu')(h_in_counter)
yc = Dense(max_digits, activation='softmax')(yc)
counter_model = Model(input=h_in_counter, output=yc, name='counter')

In [6]:
# define detector model
h_in_detector = Input(shape=(1024,))
idx_in_detector = Input(shape=(max_digits,))
yl = merge([h_in_detector, idx_in_detector], mode='concat') 

yl = Dense(512, activation='relu')(yl)
yl = Dense(512, activation='relu')(yl)
yl = Dense(10, activation='softmax')(yl)

detector_model = Model(input=[h_in_detector, idx_in_detector], output=yl, name='detector')

In [7]:
# Combine the three models to construct training graph
# We're defining the training graph as a composite made of macro models. 
# This will enable us to easily retrieve these macro components and restructure them during inference.
Ximg_in = Input(shape=(image_size[0], image_size[1], 3), name='train_input_img')
Xidx_in = Input(shape=(max_digits,), name='train_input_idx')
h = vision_model(Ximg_in)
yc = counter_model(h)
yl = detector_model([h, Xidx_in])

train_graph = Model(input=[Ximg_in, Xidx_in], output=[yc, yl])
train_graph.compile(optimizer='adamax', loss=['categorical_crossentropy','categorical_crossentropy'], metrics=['accuracy'])

In [8]:
# save the entire training graph
with open('../checkpoints/model.yaml','w') as model_def:
    model_def.write(train_graph.to_yaml())

In [10]:
# resume training if instructed and weights file exists
import os.path
if resume_training and os.path.isfile(checkpoint_path):
    model.load_weights(checkpoint_path)

## Training
DynamicPlot callback displays dynamically updated training metric plots instead of default progress bars. This gives a concise but dynamic view of training.

In [11]:
%matplotlib notebook
fig,axes = plt.subplots(2,2, figsize=(13.5,8))
fig.tight_layout()
counterLossPlot = keras_utils.DynamicPlot(metrics=['counter_loss','val_counter_loss'], axes=axes[0,0])
counterAccPlot = keras_utils.DynamicPlot(metrics=['counter_acc','val_counter_acc'], axes=axes[0,1])
labelLossPlot = keras_utils.DynamicPlot(metrics=['detector_loss','val_detector_loss'], axes=axes[1,0])
labelAccPlot = keras_utils.DynamicPlot(metrics=['detector_acc','val_detector_acc'], axes=axes[1,1])

# define checkpoint callback
checkpoint = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_detector_acc', mode='max', save_best_only=True)

history = train_graph.fit([Ximg,np_utils.to_categorical(Xidx, max_digits)], 
                    [np_utils.to_categorical(ycount, max_digits),np_utils.to_categorical(ylabel, 10)], 
                    nb_epoch=2, 
                    batch_size=64, 
                    shuffle=True, 
                    validation_split=0.05,
                    verbose=1,
                    callbacks=[counterLossPlot,counterAccPlot,labelLossPlot,labelAccPlot,
                               checkpoint])

<IPython.core.display.Javascript object>

Train on 2092 samples, validate on 111 samples
Epoch 1/2
2092/2092 [==============================] - 33s - loss: 3.4896 - counter_loss: 1.2390 - detector_loss: 2.2505 - counter_acc: 0.4622 - detector_acc: 0.1855 - val_loss: 3.3324 - val_counter_loss: 1.1132 - val_detector_loss: 2.2192 - val_counter_acc: 0.4595 - val_detector_acc: 0.1622
Epoch 2/2
2092/2092 [==============================] - 31s - loss: 3.3412 - counter_loss: 1.1457 - detector_loss: 2.1955 - counter_acc: 0.4919 - detector_acc: 0.1936 - val_loss: 3.3592 - val_counter_loss: 1.1481 - val_detector_loss: 2.2111 - val_counter_acc: 0.4595 - val_detector_acc: 0.1622
